# "Unraveling the Origins: Investigating the Cause of a Gastrointestinal Infection Outbreak and Characterizing the Properties of a Novel Pathogenic Strain"

Project #3

*Lab Journal by Artem Vasilev and Tatiana Lisitsa*

---

## Preparing

Update packages:

`$ sudo apt update && sudo apt upgrade`

Create virtual environment from `environment.yaml` file:

`$ mamba env create -f environment.yaml -p /home/<user>/<anaconda3 or conda>/envs/<env_name>`

Specify your username, an install path and name of environment

Activate it:

`$ mamba activate <env_name>`

**!WARNING!**

For the last steps of data analysis, you need the **Mauve** program, which can be problematic to run on Linux, so we recommend using <u>Windows</u> if possible

**For Windows:**

Download it from [official website](https://darlinglab.org/mauve/download.html)

**For Linux:**

`$ wget -P tools/ https://darlinglab.org/mauve/snapshots/2015/2015-02-13/linux-x64/mauve_linux_snapshot_2015-02-13.tar.gz`

`$ tar -xvzf tools/mauve_linux_snapshot_2015-02-13.tar.gz -C tools/ && rm tools/mauve_linux_snapshot_2015-02-13.tar.gz`

`$ tools/mauve_snapshot_2015-02-13/Mauve`  # didn't work for us

`$ java -jar tools/mauve_snapshot_2015-02-13/Mauve.jar`  # also didn't work

---

## Analyzing data

Many of the steps are done using **SnakeMake**. For details see `Snakefile`'s contents

---

### Download libraries and make **FastQC** reports

```
$ snakemake --cores=all -p \
results/fastqc/forward_PE_470bp_fastqc.html \
results/fastqc/reverse_PE_470bp_fastqc.html \
results/fastqc/forward_MP_2kb_fastqc.html \
results/fastqc/reverse_MP_2kb_fastqc.html \
results/fastqc/forward_MP_6kb_fastqc.html \
results/fastqc/reverse_MP_6kb_fastqc.html
```

`-p`  # print detailed output

Three libraries from the **TY2482 sample** with the following insert sizes and orientation:

**SRR292678 - paired end, insert size 470 bp (400 Mb each):**
- [forward reads](https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292678sub_S1_L001_R1_001.fastq.gz)
- [reverse reads](https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292678sub_S1_L001_R2_001.fastq.gz)

**SRR292862 – mate pair, insert size 2 kb (200 Mb each):**
- [forward reads](https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292862_S2_L001_R1_001.fastq.gz)
- [reverse reads](https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292862_S2_L001_R2_001.fastq.gz)

**SRR292770 – mate pair, insert size 6 kb (200 Mb each):**
- [forward reads](https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292770_S1_L001_R1_001.fastq.gz)
- [reverse reads](https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292770_S1_L001_R2_001.fastq.gz)

---

### Make **MultiQC** report

```
$ snakemake --cores=all -p results/multiqc/multiqc_report.html
```

After running these commands, your repository will have the following structure:
```
-/Practice/Project_3/
 |- libraries
       |- forward_MP_2kb.fastq.gz
       |- forward_MP_6kb.fastq.gz
       |- forward_PE_470bp.fastq.gz
       |- reverse_MP_2kb.fastq.gz
       |- reverse_MP_6kb.fastq.gz
       |- reverse_PE_470bp.fastq.gz
 |- results
       |- fastqc
            |- 12 items (5.2 MB)
       |- multiqc
            |- multiqc_data
                 |- 7 items (328 kB)
            |- multiqc_report.html (1.3 MB)
```

---

### Count k-mers with **Jellyfish count**

`snakemake --cores=all -p results/jellyfish/PE_470bp_kmer_31.jf`

After that change params in Snakefile!!! (`-s 500M` to `-s 260M`):

`snakemake --cores=all -p results/jellyfish/MP_2kb_kmer_31.jf`

`snakemake --cores=all -p results/jellyfish/MP_6kb_kmer_31.jf`



### Make a histogram files with **Jellyfish histo**

`snakemake --cores=all -p results/jellyfish/PE_470bp_kmer_31_histo.txt`

`snakemake --cores=all -p results/jellyfish/MP_2kb_kmer_31_histo.txt`

`snakemake --cores=all -p results/jellyfish/MP_6kb_kmer_31_histo.txt`

After running these commands, your repository will have the following structure:
```
-/Practice/Project_3/
 ...
 |- results
       ...
       |- jellyfish
            |- MP_2kb_kmer_31.jf
            |- MP_2kb_kmer_31_histo.txt
            |- MP_6kb_kmer_31.jf
            |- MP_6kb_kmer_31_histo.txt
            |- PE_470bp_kmer_31.jf
            |- PE_470bp_kmer_31_histo.txt
```

---

### Visualise histogram files with **python script**

```
$ mv visualisation.py results/jellyfish \
&& cd results/jellyfish \
&& python visualisation.py \
&& cd ../..
```

After running these commands, your repository will have the following structure:
```
-/Practice/Project_3/
 ...
 |- results
       ...
       |- jellyfish
            |- ...
            |- MP_2kb_kmer_31_histo.png
            |- MP_6kb_kmer_31_histo.png
            |- PE_470bp_kmer_31_histo.png
            |- visualisation.py
```

---

### Assemble genome with **SPAdes**

Assemble a single library of sequencing (paired end) reads:

```
spades.py -1 libraries/forward_PE_470bp.fastq.gz -2 libraries/reverse_PE_470bp.fastq.gz -o results/spades/pe_lib
```

Estimate the genome size:

```
M = 64  # k-mer peak
K = 31  # k-mer size
L = 90  # avg read length
T = 5499346*90  # total bases
N = (M*L)/(L-K+1)  # depth of coverage
genome_size = T/N
print(round(genome_size))  # 5_155_637
```

Run SPAdes again by consolidating **three libraries**:

```
spades.py -t 16 \
--pe1-1 libraries/forward_PE_470bp.fastq.gz --pe1-2 libraries/reverse_PE_470bp.fastq.gz \
--mp1-1 libraries/forward_MP_2kb.fastq.gz --mp1-2 libraries/reverse_MP_2kb.fastq.gz \
--mp2-1 libraries/forward_MP_6kb.fastq.gz --mp2-2 libraries/reverse_MP_6kb.fastq.gz \
-o results/spades/all_libs
```

After running these commands, your repository will have the following structure:
```
-/Practice/Project_3/
 ...
 |- results
       ...
       |- spades
            |- all_libs
            |- pe_lib
```

All folders contain 7 folders and 17 and 15 other items respectively. Pay attention to `contigs.fasta` and `scaffolds.fasta`

---

### Assess the quality of the resulting assembly with **Quast**

For pe_lib:

```
$ quast.py results/spades/pe_lib/contigs.fasta -o results/quast/contigs_pe_lib
$ quast.py results/spades/pe_lib/scaffolds.fasta -o results/quast/scaffolds_pe_lib
```

For all_libs:

```
$ quast.py results/spades/all_libs/contigs.fasta -o results/quast/contigs_all_libs
$ quast.py results/spades/all_libs/scaffolds.fasta -o results/quast/scaffolds_all_libs
```

After running these commands, your repository will have the following structure:
```
-/Practice/Project_3/
 ...
 |- results
       ...
       |- quast
            |- contigs_all_libs
            |- contigs_pe_lib
            |- scaffolds_all_libs
            |- scaffolds_pe_lib
```

All folders contain 2 folders, 10 other items. Pay attention to `report.html`

---

### Gene prediction and annotation with **Prokka**

`$ --centre C --locustag L prokka results/spades/all_libs/scaffolds.fasta -o results/prokka`

Two additional flags are needed for correct results interpretation in the next steps

After running this command, your repository will have the following structure:
```
-/Practice/Project_3/
 ...
 |- results
       ...
       |- prokka
            |- 12 items (56.7 MB)
```

Pay attention to `PROKKA_<date>.gbk`

---

### Find the 16S rRNA in the results of the most recent run of SPAdes with **Barrnap**

`$ mkdir results/barrnap`

`$ barrnap results/spades/all_libs/scaffolds.fasta --outseq results/barrnap/barr.fna`

After running this command, your repository will have the following structure:
```
-/Practice/Project_3/
 ...
 |- results
       ...
       |- barrnap
            |- barr.fna (30.4 kB)
```

---

### Finding the closest relative of *E. coli X* with **BLAST**

1. Go to the [BLAST page](https://blast.ncbi.nlm.nih.gov/Blast.cgi)
2. Select **Nucleotide BLAST**
3. In the **Enter Query Sequence** paste all found 16S_rRNA sequences (8) from `barr.fna`
4. Change parameters:
  - Database: RefSeq Genome Database (refseq_genomes)
  - Organism: Escherichia coli (taxid:562)
  - Entrez Query: 1900/01/01:2011/01/01[PDAT]
5. BLAST

Download the closest relative to the *E. coli X* that can be used as a reference genome:

`$ snakemake --cores=all -p reference/55989.fasta`

After running this command, your repository will have the following structure:
```
-/Practice/Project_3/
 ...
 |- results
       ...
       |- reference
            |- 55989.fasta (5.2 MB)
```

---

### Compare the *E. coli X* with the reference genome with **Mauve**

1. Remember the WARNING
2. Open `Mauve.jar`
3. `"File"` → `"Align with progressiveMauve"`
4. `"Add sequence"` and select the reference genome (`55989.fasta`) and the annotated *E. coli X* genome (`PROKKA_<date>.gbk`)
5. Manually set the `Output`:
  - Create folder for results
  - Copy its path
  - Paste to the output field and add `\<name_without_extension>`
6. Start the alignment (4 files will be created)
7. Analyse the results:
  - Use `"Sequence Navigator"` (the icon of the binoculars in the upper line)
  - Select `"Product"` in the left window and enter the name of the desired gene (`Shiga toxin`)

---

### Search for genes responsible for antibiotic resistance with [ResFinder](https://cge.food.dtu.dk/services/ResFinder/)

1. Choose file `all_libs/scaffolds.fasta`
2. Pay attention to the column named **WGS-predicted phenotype** and **Resistant** value
3. Find genes from corresponding classes in *E. coli X* via Mauve (e.g. Product = aminoglycoside)